In [9]:
import os
import numpy as np
import pandas as pd

RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
CLUSTER_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/SNPMMSeqCluster/"
RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"

In [2]:
detail_info = pd.read_excel(os.path.join(RAW_PATH, "Detail_information_344.xlsx"))

vl_samp_list = detail_info[detail_info["Types"]=='V-landrace']["Genotype No."].to_list()
g_samp_list = detail_info[detail_info["Types"]=='Grain']["Genotype No."].to_list()
vc_samp_list = detail_info[detail_info["Types"]=='V-cultivar']["Genotype No."].to_list()

output_file = pd.DataFrame(columns=["Haplotype_id", "Haplotype_class", "Chr", "Start", "End", 
                                    "Genotyped_total_num", "Genotyped_sample_num", "Genotyped_prop", 
                                    "VL_total_num", "VL_sample_num", "VL_prop",
                                    "G_total_num", "G_sample_num", "G_prop",
                                    "VC_total_num", "VC_sample_num", "VC_prop"])

file_list = [x for x in os.listdir(CLUSTER_PATH) if x.endswith(".final")]
for file_name in file_list:
    item_list = file_name.split("_")
    Chr = item_list[0]
    Start = item_list[1]
    End = item_list[2]

    region_index = "{}_{}_{}".format(Chr, Start, End)
    mmseq_res_path = os.path.join(CLUSTER_PATH, file_name)
    cluster_df = pd.read_table(mmseq_res_path)
    for group in cluster_df.groupby("Class_Rep"):
        group_name, group_data = group
        group_sample_list = group_data["Sample_name"].to_list()
        
        num = group_data.shape[0]
        total = cluster_df.shape[0]
        percentage = num / total
        if percentage <= 0.25:
            geno_pref = "NH"
        elif percentage <= 0.5:
            geno_pref = "VH"
        elif percentage <= 0.75:
            geno_pref = "IH"
        else:
            geno_pref = "PH"
        
        hap_id = "{}_{}_{}_{}".format(region_index, num, geno_pref, group_name)
        #VL
        group_vl_list = list(set(vl_samp_list).intersection(group_sample_list))
        vl_num = len(group_vl_list)
        
        total_vl_samp = list(set(vl_samp_list).intersection(list(cluster_df["Sample_name"].unique())))
        vl_total = len(total_vl_samp)
        
        vl_prop = vl_num / vl_total
        if vl_prop <= 0.25:
            vl_pref = "VLNH"
        elif vl_prop <= 0.5:
            vl_pref = "VLVH"
        elif vl_prop <= 0.75:
            vl_pref = "VLIH"
        else:
            vl_pref = "VLPH"
        # G
        group_g_list = list(set(g_samp_list).intersection(group_sample_list))
        g_num = len(group_g_list)
        
        total_g_samp = list(set(g_samp_list).intersection(list(cluster_df["Sample_name"].unique())))
        g_total = len(total_g_samp)
        
        g_prop = g_num / g_total
        if g_prop <= 0.25:
            g_pref = "GNH"
        elif g_prop <= 0.5:
            g_pref = "GVH"
        elif g_prop <= 0.75:
            g_pref = "GIH"
        else:
            g_pref = "GPH"
        #VC
        group_vc_list = list(set(vc_samp_list).intersection(group_sample_list))
        vc_num = len(group_vc_list)
        
        total_vc_samp = list(set(vc_samp_list).intersection(list(cluster_df["Sample_name"].unique())))
        vc_total = len(total_vc_samp)
        
        vc_prop = vc_num / vc_total
        if vc_prop <= 0.25:
            vc_pref = "VCNH"
        elif vc_prop <= 0.5:
            vc_pref = "VLVH"
        elif vc_prop <= 0.75:
            vc_pref = "VCIH"
        else:
            vc_pref = "VCPH"

        haplotype_class = "{}-{}-{}-{}".format(geno_pref, vl_pref, g_pref, vc_pref)
        output_file.loc[len(output_file)] = [hap_id, haplotype_class, Chr, Start, End, 
                                             total, num, percentage, 
                                             vl_total, vl_num, vl_prop,
                                             g_total, g_num, g_prop, 
                                             vc_total, vc_num, vc_prop]

output_file.to_csv(os.path.join(RES_PATH, "Haplotype_Counts_Info.tsv"), sep="\t", index=0)
print("ALL DONE")

ALL DONE


In [3]:
output_file

,Haplotype_id,Haplotype_class,Chr,Start,End,Genotyped_total_num,Genotyped_sample_num,Genotyped_prop,VL_total_num,VL_sample_num,VL_prop,G_total_num,G_sample_num,G_prop,VC_total_num,VC_sample_num,VC_prop
0,Chr01_51400001_51530000_218_IH_D367,IH-VLIH-GVH-VCPH,Chr01,51400001,51530000,332,218,0.656627,159,106,0.666667,82,32,0.390244,79,73,0.924051
1,Chr01_51400001_51530000_1_NH_D403,NH-VLNH-GNH-VCNH,Chr01,51400001,51530000,332,1,0.003012,159,1,0.006289,82,0,0.000000,79,0,0.000000
2,Chr01_51400001_51530000_2_NH_D610,NH-VLNH-GNH-VCNH,Chr01,51400001,51530000,332,2,0.006024,159,2,0.012579,82,0,0.000000,79,0,0.000000
3,Chr01_51400001_51530000_1_NH_D614,NH-VLNH-GNH-VCNH,Chr01,51400001,51530000,332,1,0.003012,159,1,0.006289,82,0,0.000000,79,0,0.000000
4,Chr01_51400001_51530000_47_NH_D628,NH-VLNH-GNH-VCNH,Chr01,51400001,51530000,332,47,0.141566,159,31,0.194969,82,10,0.121951,79,4,0.050633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12647,Chr09_12710001_12950000_1_NH_D717,NH-VLNH-GNH-VCNH,Chr09,12710001,12950000,338,1,0.002959,164,0,0.000000,81,1,0.012346,80,0,0.000000
12648,Chr09_12710001_12950000_2_NH_D722,NH-VLNH-GNH-VCNH,Chr09,12710001,12950000,338,2,0.005917,164,0,0.000000,81,2,0.024691,80,0,0.000000
12649,Chr09_12710001_12950000_1_NH_D724,NH-VLNH-GNH-VCNH,Chr09,12710001,12950000,338,1,0.002959,164,0,0.000000,81,1,0.012346,80,0,0.000000
12650,Chr09_12710001_12950000_1_NH_D725,NH-VLNH-GNH-VCNH,Chr09,12710001,12950000,338,1,0.002959,164,0,0.000000,81,0,0.000000,80,0,0.000000


In [10]:
raw_info = pd.read_table(os.path.join(RES_PATH, "Haplotype_Counts_Info.tsv"))

In [ ]:
a = raw_info[['Chr', 'Start', 'End']].drop_duplicates()
a[a["Chr"]=="Chr11"].sort_values(by=['Start', 'End'])

In [3]:
raw_info = pd.read_table(os.path.join(RES_PATH, "Haplotype_Counts_Info.tsv"))
raw_info.to_excel(os.path.join(RES_PATH, "Haplotype_Counts_Info.xlsx"), index=0)

(12652, 17)